# Load Model

In [1]:
!wget https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Utility/Model/BBR_MDistilBERT.pkl

--2021-08-05 21:21:44--  https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Utility/Model/BBR_MDistilBERT.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110300 (2.0M) [application/octet-stream]
Saving to: ‘BBR_MDistilBERT.pkl’

BBR_MDistilBERT.pkl 100%[===================>]   2.01M  --.-KB/s    in 0.1s    

2021-08-05 21:21:44 (21.0 MB/s) - ‘BBR_MDistilBERT.pkl’ saved [2110300/2110300]



In [2]:
pkl_filename = "BBR_MDistilBERT.pkl"

In [3]:
import pickle
with open(pkl_filename, 'rb') as file:
  rgs = pickle.load(file)

# Represent User Text

In [4]:
!pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 2.6 MB 38.0 MB/s 
     |████████████████████████████████| 1.2 MB 26.1 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 636 kB 50.2 MB/s 
     |████████████████████████████████| 895 kB 54.7 MB/s 
     |████████████████████████████████| 3.3 MB 52.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=7b7e37190a86912683b42da5d29ac867ab7b17f316aed34eb37bf55fb4cc496b
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [6]:
import pandas as pd
def WordEmbeddings(texts):

  model = SentenceTransformer('distiluse-base-multilingual-cased')

  if type(texts) == pd.core.series.Series:
    sentences = texts.replace(['\t','\n','\r'], [' ',' ',' '], regex=True)
  else:
    sentences = texts

  sentence_embeddings = model.encode(list(sentences))
  
  return sentence_embeddings

The variable *txts* can be a dataframe column or a list of strings

# DEMO


In [21]:
df = pd.DataFrame(columns=['text'])
df = df.append({'text': 'the app always crashes !!!!!!!!!!'},ignore_index=True)
df = df.append({'text': 'I loved this app!!'},ignore_index=True)

data = WordEmbeddings(df.text)

predicted_utilites = rgs.predict(data)
predicted_utilites

array([0.00210714, 0.00201899])

# Utility Metrics

In [8]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

In [23]:
y_test_utility = [1, 0]

## Mean Absolute Error

In [24]:
mae(y_test_utility,predicted_utilites)

0.4999559247204315

## Mean Squared Error

In [25]:
mse(y_test_utility,predicted_utilites)

0.4978971171726119

## r2

In [26]:
r2(y_test_utility,predicted_utilites)

-0.9915884686904477